Importing libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


data loading and preprocessing

In [ ]:
df=pd.read_csv(r'C:\Users\kmmor\Desktop\Finlatics\ML\Projects\Facebook Dataset\Facebook_Marketplace_data.csv',parse_dates=['status_published'])
print(df.info())
df.drop(columns=["Column1","Column2","Column3","Column4"],axis=1,inplace=True)
print(df.columns)

Data analysis

1.	How does the time of upload (`status_published`)  affects the `num_reaction`?

In [ ]:
df['Hour_published']=df['status_published'].dt.hour
df['Week_published']=df['status_published'].dt.dayofweek

#Linear relationship checking
corr_matrix=df[['Hour_published','Week_published','num_reactions']].corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr_matrix,annot=True,cmap="PuBuGn",fmt=".2f")
plt.title("Correlation matrix between time of publish and number of reactions")
plt.show()

#For each hour of a day
hr=df.groupby("Hour_published")['num_reactions'].mean().reset_index()
print(hr)
plt.plot(hr['Hour_published'],hr['num_reactions'])
plt.xlabel("Hour of a day")
plt.ylabel("Average number of reactions")
plt.title("Average reactions of an hour in a day")
plt.xticks(range(0,24))
plt.show()

#For which day and hour
hr_day=df.groupby(["Week_published","Hour_published"])['num_reactions'].mean().unstack()
t1=hr_day
t2=hr_day.values
plt.figure()
plt.imshow(hr_day, aspect='auto', origin='lower')
plt.colorbar(label='Average Number of Reactions')
plt.xlabel('Hour of Day')
plt.ylabel('Day of Week')
plt.title('Average Reactions by Day of Week and Hour')
plt.xticks(range(0, 24))
plt.yticks(range(0, 7), ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
plt.show()

2.	Is there a correlation between the number of reactions (num_reactions) and other engagement metrics such as comments (num_comments) and shares (num_shares)? If so, what is the strength and direction of this correlation?

In [ ]:
corr_matrix2=df[['num_reactions','num_comments','num_shares']].corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr_matrix2,annot=True,cmap="PuBuGn",fmt=".2f")
plt.title("Correlation between number of reactions and number of shares & comments")
plt.show()

 3.	Use the columns status_type, num_reactions, num_comments, num_shares, num_likes, num_loves, num_wows, num_hahas, num_sads, and num_angrys to train a K-Means clustering model on the Facebook Live Sellers dataset.

In [ ]:
print(df.shape)
ds3=df[df.columns[np.r_[1,3:12]]]
print(ds3['status_type'].unique())

 #Data encoding for categorical variables
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

oe=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[0])],remainder='passthrough')
ds3=oe.fit_transform(ds3)
 #Feature scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
ds3[:,4:]=sc.fit_transform(ds3[:,4:])

 #Model fitting
from sklearn.cluster import KMeans
wcss=[]
for i in range(1,30):
    kmeans=KMeans(n_clusters=i,init='k-means++',random_state=50)
    kmeans.fit(ds3)
    wcss.append(kmeans.inertia_)
plt.plot(range(1,30),wcss)
plt.title("wcss vs number of clusters")
plt.xlabel("Number of clusters (k)")
plt.ylabel("WCSS")
plt.show()

kmeans=KMeans(n_clusters=12,init='k-means++',random_state=50)
kmeans_pred=kmeans.fit_predict(ds3)

4.	Use the elbow method to find the optimum number of clusters

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

oe=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[0])],remainder='passthrough')
ds3=oe.fit_transform(ds3)
 #Feature scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
ds3[:,4:]=sc.fit_transform(ds3[:,4:])

 #Model fitting
from sklearn.cluster import KMeans
wcss=[]
for i in range(1,30):
    kmeans=KMeans(n_clusters=i,init='k-means++',random_state=50)
    kmeans.fit(ds3)
    wcss.append(kmeans.inertia_)
plt.plot(range(1,30),wcss)
plt.title("wcss vs number of clusters")
plt.xlabel("Number of clusters (k)")
plt.ylabel("WCSS")
plt.show()

5. What is the count of different types of posts in the dataset?

In [ ]:
n_status_types=df['status_type'].value_counts()
print(n_status_types)

	6. What is the average value of num_reaction, num_comments, num_shares for each post type?

In [ ]:
Avg_num_reaction=round(df.groupby(['status_type'])['num_reactions'].mean(),2)
print(Avg_num_reaction)

Avg_num_comments=round(df.groupby(['status_type'])['num_comments'].mean(),2)
print(Avg_num_comments)

Avg_num_shares=round(df.groupby(['status_type'])['num_shares'].mean(),2)
print(Avg_num_shares)